In [9]:
from mlflow.tracking import MlflowClient
from mlflow.entities import ViewType


MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"
client = MlflowClient(tracking_uri=MLFLOW_TRACKING_URI)

In [66]:
import mlflow

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("nyc-taxi-experiment")

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-training/mlruns/1', creation_time=1742114042146, experiment_id='1', last_update_time=1742114042146, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>

In [74]:
experiments = client.search_experiments()
for exp in experiments:
    print(exp)

<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-training/mlruns/3', creation_time=1742365887040, experiment_id='3', last_update_time=1742365887040, lifecycle_stage='active', name='my-cool-new-experiment', tags={}>
<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-training/mlruns/2', creation_time=1742362376397, experiment_id='2', last_update_time=1742362376397, lifecycle_stage='active', name='my-cool-experiment', tags={}>
<Experiment: artifact_location='/workspaces/mlops-zoomcamp/02-training/mlruns/1', creation_time=1742114042146, experiment_id='1', last_update_time=1742114042146, lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>
<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1742113900835, experiment_id='0', last_update_time=1742113900835, lifecycle_stage='active', name='Default', tags={}>


In [77]:
client.create_experiment(name="my-one-cool-new-experiment")
print('ok')

ok


In [49]:
runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 7",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)

In [50]:
for run in runs:
    print(f"run id: {run.info.run_id}, rmse: {run.data.metrics['rmse']:.4f}")

run id: 375c3d4d3df84ed287912fc09e252eb0, rmse: 6.3184
run id: 94579fdc77bf4719888fa5b10345a6fb, rmse: 6.3184
run id: 5c1eff8eb9614758be67a790e70891ff, rmse: 6.4013
run id: 03bcbc2aa4f14003831383bb140f24c9, rmse: 6.5417


In [12]:
import mlflow

mlflow.set_tracking_uri(MLFLOW_TRACKING_URI)

In [51]:
run_id = "375c3d4d3df84ed287912fc09e252eb0"
model_uri = f"runs:/{run_id}/model"
mlflow.register_model(model_uri=model_uri, name="nyc-taxi")

Registered model 'nyc-taxi' already exists. Creating a new version of this model...
Created version '6' of model 'nyc-taxi'.


<ModelVersion: aliases=[], creation_timestamp=1742365141863, current_stage='None', description=None, last_updated_timestamp=1742365141863, name='nyc-taxi', run_id='375c3d4d3df84ed287912fc09e252eb0', run_link=None, source='/workspaces/mlops-zoomcamp/02-training/mlruns/1/375c3d4d3df84ed287912fc09e252eb0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [52]:
model_uri

'runs:/375c3d4d3df84ed287912fc09e252eb0/model'

In [53]:
model_name = "nyc-taxi"
latest_versions = client.get_latest_versions(name=model_name)

for version in latest_versions:
    print(f"version: {version.version}, stage: {version.current_stage}")

version: 3, stage: Staging
version: 6, stage: None
version: 4, stage: Production


/tmp/ipykernel_7596/574510856.py:2: FutureWarning: ``mlflow.tracking.client.MlflowClient.get_latest_versions`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  latest_versions = client.get_latest_versions(name=model_name)


In [54]:
model_version = 6
new_stage = "Staging"
client.transition_model_version_stage(
    name=model_name,
    version=model_version,
    stage=new_stage,
    archive_existing_versions=False
)

/tmp/ipykernel_7596/3165129868.py:3: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1742365141863, current_stage='Staging', description=None, last_updated_timestamp=1742365161203, name='nyc-taxi', run_id='375c3d4d3df84ed287912fc09e252eb0', run_link=None, source='/workspaces/mlops-zoomcamp/02-training/mlruns/1/375c3d4d3df84ed287912fc09e252eb0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [55]:
from datetime import datetime

date = datetime.today().date()
client.update_model_version(
    name=model_name,
    version=model_version,
    description=f"The model version {model_version} was transitioned to {new_stage} on {date}"
)

<ModelVersion: aliases=[], creation_timestamp=1742365141863, current_stage='Staging', description='The model version 6 was transitioned to Staging on 2025-03-19', last_updated_timestamp=1742365164196, name='nyc-taxi', run_id='375c3d4d3df84ed287912fc09e252eb0', run_link=None, source='/workspaces/mlops-zoomcamp/02-training/mlruns/1/375c3d4d3df84ed287912fc09e252eb0/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=6>

In [56]:
from sklearn.metrics import mean_squared_error
import pandas as pd


def read_dataframe(filename):
    df = pd.read_parquet(filename)

    df.lpep_dropoff_datetime = pd.to_datetime(df.lpep_dropoff_datetime)
    df.lpep_pickup_datetime = pd.to_datetime(df.lpep_pickup_datetime)

    df['duration'] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    
    return df


def preprocess(df, dv):
    df['PU_DO'] = df['PULocationID'] + '_' + df['DOLocationID']
    categorical = ['PU_DO']
    numerical = ['trip_distance']
    train_dicts = df[categorical + numerical].to_dict(orient='records')
    return dv.transform(train_dicts)


def test_model(name, stage, X_test, y_test):
    model = mlflow.pyfunc.load_model(f"models:/{name}/{stage}")
    y_pred = model.predict(X_test)
    return {"rmse": mean_squared_error(y_test, y_pred, squared=False)}

In [57]:
df = read_dataframe("data/green_tripdata_2021-03.parquet")

In [29]:
df.head()

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,duration
0,2,2021-03-01 00:05:42,2021-03-01 00:14:03,N,1.0,83,129,1.0,1.56,7.5,...,0.5,0.00,0.0,None,0.3,8.80,1.0,1.0,0.0,8.350000
1,2,2021-03-01 00:21:03,2021-03-01 00:26:17,N,1.0,243,235,1.0,0.96,6.0,...,0.5,0.00,0.0,None,0.3,7.30,2.0,1.0,0.0,5.233333
2,2,2021-03-01 00:02:06,2021-03-01 00:22:26,N,1.0,75,242,1.0,9.93,28.0,...,0.5,2.00,0.0,None,0.3,31.30,1.0,1.0,0.0,20.333333
3,2,2021-03-01 00:24:03,2021-03-01 00:31:43,N,1.0,242,208,1.0,2.57,9.5,...,0.5,0.00,0.0,None,0.3,10.80,2.0,1.0,0.0,7.666667
4,1,2021-03-01 00:11:10,2021-03-01 00:14:46,N,1.0,41,151,1.0,0.80,5.0,...,0.5,1.85,0.0,None,0.3,8.15,1.0,1.0,0.0,3.600000


In [58]:
client.download_artifacts(run_id=run_id, path='preprocessor', dst_path='.')


'/workspaces/mlops-zoomcamp/02-training/preprocessor'

In [60]:
import pickle

with open("models/preprocessor.b", "rb") as f_in:
    dv = pickle.load(f_in)

In [61]:
X_test = preprocess(df, dv)

In [62]:
target = "duration"
y_test = df[target].values

In [69]:
client.transition_model_version_stage(
    name=model_name,
    version=4,
    stage="Production",
    archive_existing_versions=True
)

/tmp/ipykernel_7596/3610659809.py:1: FutureWarning: ``mlflow.tracking.client.MlflowClient.transition_model_version_stage`` is deprecated since 2.9.0. Model registry stages will be removed in a future major release. To learn more about the deprecation of model registry stages, see our migration guide here: https://mlflow.org/docs/latest/model-registry.html#migrating-from-stages
  client.transition_model_version_stage(


<ModelVersion: aliases=[], creation_timestamp=1742363055630, current_stage='Production', description='', last_updated_timestamp=1742365793182, name='nyc-taxi', run_id='03bcbc2aa4f14003831383bb140f24c9', run_link='', source='models:/nyc-taxi/3', status='READY', status_message=None, tags={}, user_id=None, version=4>

In [78]:
print('workshop done')

workshop done


In [80]:
print('testing')

testing
